In [ ]:
from enum import Enum
from PIL import Image
from typing import Optional, Tuple
from scipy.ndimage import gaussian_filter, correlate
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import os

In [ ]:
class FailReason(Enum):
    DIMENSIONS, GRAYSCALE, BACKGROUND = range(3)


def to_grayscale(im: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    return im @ np.asarray([0.299, 0.587, 0.114])


def gaussian_derivative(col_values: npt.NDArray[np.float64], sigma: float) -> npt.NDArray[np.float64]:
    row_values = col_values.T
    return row_values / (sigma**3 * np.sqrt(2 * np.pi)) * np.exp(-(row_values**2 + col_values ** 2)/ (2 * sigma**2))


def gaussian_derivative_mask(sigma: float) -> npt.NDArray[np.float64]:
    """
    Gaussian derivative mask for the rows (transpose for columns).
    """
    center_idx = int(np.ceil(2 * sigma))
    mask_size = 2 * center_idx + 1
    col_values = np.zeros(shape=(mask_size, mask_size))
    col_values[:] = np.linspace(-center_idx, center_idx, 2 * center_idx + 1)

    return gaussian_derivative(col_values, sigma)


def shrink_im(im: npt.NDArray[np.float64], max_height: int) -> npt.NDArray[np.float64]:
    print('hello')
    shrinked_im = im.copy()
    while shrinked_im.shape[0] > max_height:
        shrinked_im = gaussian_filter(shrinked_im, sigma=1, axes=(0, 1))
        shrinked_im = shrinked_im[::2, ::2, ...]
    
    return shrinked_im


class PassbildVerifier:
    EDGE_THRESHOLD = 0.1
    def __init__(self, im: npt.NDArray[np.float64]):
        self.im = im


    def correct_dimensions(self) -> bool:
        correct_width = np.round(self.im.shape[0] / 45 * 35)

        return self.im.shape[1] == correct_width


    def is_color_image(self) -> bool:
        try:
            is_rgb = self.im.shape[2] == 3
        except IndexError:
            is_rgb = False

        return is_rgb and not (self.im[..., 0] == self.im[..., 1]).all() and not (self.im[..., 1] == self.im[..., 2]).all()
    

    def solid_background(self) -> bool:
        background_region_width = int(self.edges.shape[1] * 0.15)
        background_region_height = int(self.edges.shape[0] * 0.6)

        background_mask = np.zeros(self.edges.shape, dtype=bool)
        background_mask[:background_region_height, :background_region_width] = True
        background_mask[:background_region_height, -background_region_width:] = True

        corner_color_diff = np.max(np.abs(self.shrinked_im[0, 0] - self.shrinked_im[0, -1]))
    
        return corner_color_diff < 0.2 and not ((self.edges > self.EDGE_THRESHOLD) & background_mask).any()


    def verify(self) -> bool:
        checks = {
            self.correct_dimensions: FailReason.DIMENSIONS,
            self.is_color_image: FailReason.GRAYSCALE,
            self.solid_background: FailReason.BACKGROUND,
        }

        for check_func, fail_reason in checks.items():
            if not check_func():
                return False, fail_reason
        
        return True, None


    @property
    def shrinked_im(self) -> npt.NDArray[np.float64]:
        try:
            return self._shrinked_im
        except AttributeError:
            self._shrinked_im = shrink_im(self.im, 128)
            return self._shrinked_im
        
    
    @property
    def shrinked_greyscale_im(self) -> npt.NDArray[np.float64]:
        return to_grayscale(self.shrinked_im)
    

    @property
    def edges(self) -> npt.NDArray[np.float64]:
        try:
            return self._edges
        except AttributeError:
            derivative_mask = gaussian_derivative_mask(1)
            self._edges = np.sqrt(correlate(self.shrinked_greyscale_im, derivative_mask) ** 2 + correlate(self.shrinked_greyscale_im, derivative_mask.T) ** 2)
            return self._edges



def is_passbild(im: npt.NDArray) -> Tuple[bool, Optional[FailReason]]:
    return PassbildVerifier(im).verify()

In [ ]:
INPUT_DIR = './inputs/'

for filename in os.listdir(INPUT_DIR):
    filepath = os.path.join(INPUT_DIR, filename)

    try:
        im = np.array(Image.open(filepath), dtype=np.float64) / 255
    except IOError:
        continue

    reason_message = {
        FailReason.DIMENSIONS: 'Ein richtiges Passbild muss 45mm hoch und 35mm breit sein.',
        FailReason.GRAYSCALE: 'Ein richtiges Passbild muss in Farbe sein.',
        FailReason.BACKGROUND: 'Ein richtiges Passbild muss einen einfarbigen Hintergrund haben.'
    }

    result, reason = is_passbild(im)

    if result:
        title = 'Passbild :)'
        message = 'Das ist ein Passbild!'
    else:
        title = 'Kein Passbild :('
        message = f'Das ist leider kein Passbild! {reason_message[reason]}'

    # print(gaussian_derivative_mask(1))

    derivative_mask = gaussian_derivative_mask(1)

    # im = shrink_im(im.astype(np.float64) / 255, 50)

    plt.imshow(im, cmap='gray')
    plt.axis('off')
    plt.title(title)

    plt.show()

    print(message)